1. Globale Middleware in Authentifizierung einbauen 
2. Spezifische Middleware = Policies für Routen




#### Middleware in Strapi
#### Was ist Middleware?
Middleware ist eine Funktion, die zwischen einer eingehenden HTTP-Anfrage und der endgültigen Verarbeitung dieser Anfrage durch den Controller oder die Route steht. Sie kann verwendet werden, um allgemeine Aufgaben wie Authentifizierung, Logging, Fehlerbehandlung und mehr durchzuführen.

##### Arten von Middleware
###### Globale Middleware:

* Wird auf alle Anfragen angewendet.
* Wird in der globalen Middleware-Konfiguration registriert.
Beispiele: Authentifizierung, allgemeine Sicherheitsmaßnahmen, Logging.

###### Spezifische Middleware:

* Kann für bestimmte Routen oder Content Types konfiguriert werden.
* Bietet granularere Kontrolle.
Beispiele: Spezifische Validierungen, Ratenbegrenzung für bestimmte Endpunkte.

##### Einsatz von Middleware

###### Globale Authentifizierung:
* Eine globale Middleware, die sicherstellt, dass alle Anfragen authentifiziert sind.

In [2]:
// ./middlewares/auth.js
module.exports = async (ctx, next) => {
    if (ctx.state.user) {
      await next();
    } else {
      ctx.unauthorized(`You're not logged in!`);
    }
  };
  
  // ./config/middleware.js
  module.exports = {
    settings: {
      auth: {
        enabled: true,
      },
    },
  };

SyntaxError: unterminated string literal (detected at line 6) (1026841611.py, line 6)

###### Spezifische Middleware:
* Middleware, die nur auf bestimmte Routen angewendet wird.

In [ ]:
// ./middlewares/rateLimiter.js
module.exports = async (ctx, next) => {
    // Implementierung der Ratenbegrenzung
    await next();
  };
  
  // Registrierung in der Routing-Konfiguration
  {
    "routes": [
      {
        "method": "GET",
        "path": "/notes",
        "handler": "note.find",
        "config": {
          "policies": ["global::auth", "middlewares::rateLimiter"]
        }
      }
    ]
  }

: 

#### Policies in Strapi
#### Was sind Policies?
Policies sind Funktionen, die speziell für die Zugangskontrolle und Berechtigungen konzipiert sind. Sie werden verwendet, um zu bestimmen, ob eine Anfrage autorisiert ist, bevor sie an den Controller weitergeleitet wird. Policies sind flexibler und granularer als globale Middleware und können auf einzelne Routen oder Content Types angewendet werden.

##### Einsatz von Policies
###### Benutzerdefinierte Policy:
Eine Policy, die überprüft, ob ein Benutzer bestimmte Berechtigungen hat.

In [ ]:
// ./api/note/policies/isAzubiPolicy.js
module.exports = async (ctx, next) => {
    const user = ctx.state.user;
  
    if (!user) {
      return ctx.unauthorized(`You're not logged in!`);
    }
  
    if (user.isAzubi) {
      // Azubis sehen nur ihre eigenen Noten
      ctx.query.user = user.id;
    } else {
      // Andere Benutzer sehen alle Noten von Azubis
      const azubiUsers = await strapi.query('user', 'users-permissions').find({ isAzubi: true });
      const azubiUserIds = azubiUsers.map(azubi => azubi.id);
      ctx.query.user_in = azubiUserIds;
    }
  
    await next();
  };

: 

In [ ]:
// Registrierung in der Routing-Konfiguration
{
    "routes": [
      {
        "method": "GET",
        "path": "/notes",
        "handler": "note.find",
        "config": {
          "policies": ["global::auth", "api::note.isAzubiPolicy"]
        }
      }
    ]
  }

: 

### Gegenüberstellung: Middleware vs. Policies

### Fazit
Middleware ist ideal für allgemeine, wiederkehrende Aufgaben, die auf alle oder eine breite Palette von Anfragen angewendet werden müssen.
Policies sind spezialisierter und eignen sich hervorragend für feingranulare Zugangskontrollen und Berechtigungen auf Ebene einzelner Routen oder Content Types.
Durch die Kombination beider Mechanismen kannst du eine robuste und skalierbare Sicherheits- und Zugriffskontrollstruktur für deine Strapi-Anwendung aufbauen. Globale Middleware sorgt für grundlegende Sicherheit und Authentifizierung, während Policies für spezifische Zugriffskontrollen und Berechtigungsprüfungen sorgen.

In [8]:
%%html<div><img src="/Users/denise/sdv/offspring/projekt/image.png" width="600"> </div>